In [2]:
import pandas as pd
import numpy as np
import re

In [ ]:
df = pd.read_csv(r'C:/Users/kubad/Desktop/PROJECTS/Guvi_Project-2/Luxury_Housing_Bangalore.csv')

print(f"Original shape: {df.shape}")

df.head(10)

Original shape: (101000, 18)


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4bhk,12.750846039118798,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,yes,9.212491,18,Loved the amenities!
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3Bhk,16.292151871065954,Primary,Other,2024-06-30,4.839024,NaN,Under construction,NRI Desk,no,7.723898,106,NaN
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4bhk,10.517724412961911,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,yes,6.985493,113,Agent was not responsive.
3,PROP000004,bellary road,Project_3,Embassy,6192.0,3BHK,9.396367494232896,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,yes,6.100929,106,Excellent location!
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4Bhk,15.345392444511946,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,no,5.312510,18,Too far from my office.
5,PROP000006,YELAHANKA,Project_5,Brigade,4290.0,4bhk,₹9.82 Cr,Secondary,HNI,2023-09-30,7.936068,NaN,Launch,Online,no,5.154027,16,Underpriced for location.
6,PROP000007,Bannerghatta Road,Project_6,Prestige,6776.0,5BHK+,10.896752354445224,Secondary,Other,2025-03-31,8.643117,8.355238,Under construction,Direct,no,9.791550,74,Too far from my office.
7,PROP000008,whitefield,Project_7,Total Environment,5763.0,3BHK,9.357538943683888,Secondary,NRI,2025-03-31,6.781306,9.565013,Under construction,Direct,yes,9.054713,31,NaN
8,PROP000009,sarjapur road,Project_8,Prestige,5815.0,3Bhk,12.299449409872416,Secondary,Other,2024-03-31,7.284164,5.580442,Under construction,NRI Desk,yes,8.552046,68,NaN
9,PROP000010,JP NAGAR,Project_9,Puravankara,8675.0,4Bhk,9.193310278863123,Primary,CXO,2023-09-30,9.964399,7.759907,Launch,Direct,yes,8.395691,18,Too far from my office.


In [6]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 101000 entries, 0 to 100999
Data columns (total 18 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   Property_ID           101000 non-null  str    
 1   Micro_Market          101000 non-null  str    
 2   Project_Name          101000 non-null  str    
 3   Developer_Name        101000 non-null  str    
 4   Unit_Size_Sqft        90954 non-null   float64
 5   Configuration         101000 non-null  str    
 6   Ticket_Price_Cr       90981 non-null   str    
 7   Transaction_Type      101000 non-null  str    
 8   Buyer_Type            101000 non-null  str    
 9   Purchase_Quarter      101000 non-null  str    
 10  Connectivity_Score    101000 non-null  float64
 11  Amenity_Score         90910 non-null   float64
 12  Possession_Status     101000 non-null  str    
 13  Sales_Channel         101000 non-null  str    
 14  NRI_Buyer             101000 non-null  str    
 15  Locality_In

In [21]:

def clean_currency(x):
    if pd.isna(x):
        return np.nan
    
    clean_str = re.sub(r'[^\d.]', '', str(x))
    try:
        return float(clean_str)
    except ValueError:
        return np.nan

df['Ticket_Price_Cr'] = df['Ticket_Price_Cr'].apply(clean_currency)


df['Micro_Market'] = df['Micro_Market'].str.strip().str.title()


df['Configuration'] = df['Configuration'].str.strip().str.upper()


df.loc[df['Unit_Size_Sqft'] <= 0, 'Unit_Size_Sqft'] = np.nan
df['Unit_Size_Sqft'] = df['Unit_Size_Sqft'].fillna(
    df.groupby('Configuration')['Unit_Size_Sqft'].transform('median')
)


df['Amenity_Score'] = df['Amenity_Score'].fillna(df['Amenity_Score'].median())


df['Purchase_Quarter'] = pd.to_datetime(df['Purchase_Quarter'], errors='coerce')

df['NRI_Buyer'] = np.where(df['NRI_Buyer'].str.lower() == 'yes', 1, 0)

In [22]:
df.dropna(subset=['Ticket_Price_Cr'], inplace=True)

In [23]:

df['Price_Per_Sqft'] = (df['Ticket_Price_Cr'] * 10_000_000) / df['Unit_Size_Sqft']

In [24]:
df['Buyer_Comments'] = df['Buyer_Comments'].fillna("No Comment")

In [25]:
print("Final Cleaned Shape:", df.shape)
print("Missing Values Check:")
print(df.isnull().sum())
df.info()


Final Cleaned Shape: (90981, 19)
Missing Values Check:
Property_ID             0
Micro_Market            0
Project_Name            0
Developer_Name          0
Unit_Size_Sqft          0
Configuration           0
Ticket_Price_Cr         0
Transaction_Type        0
Buyer_Type              0
Purchase_Quarter        0
Connectivity_Score      0
Amenity_Score           0
Possession_Status       0
Sales_Channel           0
NRI_Buyer               0
Locality_Infra_Score    0
Avg_Traffic_Time_Min    0
Buyer_Comments          0
Price_Per_Sqft          0
dtype: int64
<class 'pandas.DataFrame'>
Index: 90981 entries, 0 to 100999
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Property_ID           90981 non-null  str           
 1   Micro_Market          90981 non-null  str           
 2   Project_Name          90981 non-null  str           
 3   Developer_Name        90981 non-null  str        

In [26]:
df.head()

,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Price_Per_Sqft
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4BHK,12.750846,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,1,9.212491,18,Loved the amenities!,31679.120594
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3BHK,16.292152,Primary,Other,2024-06-30,4.839024,7.499123,Under construction,NRI Desk,0,7.723898,106,No Comment,28284.985887
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4BHK,10.517724,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,1,6.985493,113,Agent was not responsive.,13646.976013
3,PROP000004,Bellary Road,Project_3,Embassy,6192.0,3BHK,9.396367,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,1,6.100929,106,Excellent location!,15175.012103
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4BHK,15.345392,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,0,5.312510,18,Too far from my office.,21471.096187


In [ ]:
df.to_csv('Cleaned_Luxury_Housing.csv', index=False)
print("SUCCESS: 'Cleaned_Luxury_Housing.csv' has been saved.")

------------------------------------------------
SUCCESS: 'Cleaned_Luxury_Housing.csv' has been saved.


In [29]:
df = pd.read_csv(r'C:/Users/kubad/Desktop/PROJECTS/Guvi_Project-2/Cleaned_Luxury_Housing.csv')

print(f"Original shape: {df.shape}")

df.head(10)

Original shape: (90981, 19)


,Property_ID,Micro_Market,Project_Name,Developer_Name,Unit_Size_Sqft,Configuration,Ticket_Price_Cr,Transaction_Type,Buyer_Type,Purchase_Quarter,Connectivity_Score,Amenity_Score,Possession_Status,Sales_Channel,NRI_Buyer,Locality_Infra_Score,Avg_Traffic_Time_Min,Buyer_Comments,Price_Per_Sqft
0,PROP000001,Sarjapur Road,Project_0,RMZ,4025.0,4BHK,12.750846,Primary,NRI,2025-03-31,7.990091,5.462863,Launch,Broker,1,9.212491,18,Loved the amenities!,31679.120594
1,PROP000002,Indiranagar,Project_1,Puravankara,5760.0,3BHK,16.292152,Primary,Other,2024-06-30,4.839024,7.499123,Under construction,NRI Desk,0,7.723898,106,No Comment,28284.985887
2,PROP000003,Bannerghatta Road,Project_2,Tata Housing,7707.0,4BHK,10.517724,Primary,HNI,2023-12-31,8.131315,8.669227,Ready to move,Direct,1,6.985493,113,Agent was not responsive.,13646.976013
3,PROP000004,Bellary Road,Project_3,Embassy,6192.0,3BHK,9.396367,Primary,HNI,2024-03-31,7.501657,5.720246,Ready to move,Online,1,6.100929,106,Excellent location!,15175.012103
4,PROP000005,Koramangala,Project_4,SNN Raj,7147.0,4BHK,15.345392,Secondary,HNI,2024-12-31,4.525216,8.609649,Under construction,Broker,0,5.312510,18,Too far from my office.,21471.096187
5,PROP000006,Yelahanka,Project_5,Brigade,4290.0,4BHK,9.820000,Secondary,HNI,2023-09-30,7.936068,7.499123,Launch,Online,0,5.154027,16,Underpriced for location.,22890.442890
6,PROP000007,Bannerghatta Road,Project_6,Prestige,6776.0,5BHK+,10.896752,Secondary,Other,2025-03-31,8.643117,8.355238,Under construction,Direct,0,9.791550,74,Too far from my office.,16081.393675
7,PROP000008,Whitefield,Project_7,Total Environment,5763.0,3BHK,9.357539,Secondary,NRI,2025-03-31,6.781306,9.565013,Under construction,Direct,1,9.054713,31,No Comment,16237.270421
8,PROP000009,Sarjapur Road,Project_8,Prestige,5815.0,3BHK,12.299449,Secondary,Other,2024-03-31,7.284164,5.580442,Under construction,NRI Desk,1,8.552046,68,No Comment,21151.245761
9,PROP000010,Jp Nagar,Project_9,Puravankara,8675.0,4BHK,9.193310,Primary,CXO,2023-09-30,9.964399,7.759907,Launch,Direct,1,8.395691,18,Too far from my office.,10597.475826
